# Glaucoma Segmentation


## Imports

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import defaultdict
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

from networks import *
from training import *
from utils import *

## Setup

In [ ]:
IMAGE_DIR = '../data/ORIGA/Images_Cropped'
MASK_DIR = '../data/ORIGA/Masks_Cropped'
LOGS_DIR = '../logs/'
CHECKPOINT_DIR = '../checkpoints/'
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
in_channels = 3
out_channels = 1
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 3
LAYERS = [32, 64, 128, 256, 512]
EARLY_STOPPING_PATIENCE = 10
LOG_INTERVAL = 1
SAVE_INTERVAL = 10
NUM_WORKERS = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PIN_MEMORY = True if DEVICE == 'cuda' else False
USE_WANDB = False
DEEP_SUPERVISION = False

os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f'''CONFIGURATION:
    PyTorch version: {torch.__version__}
    Using device: {DEVICE}
    Image directory: {IMAGE_DIR}
    Mask directory: {MASK_DIR}
    Input image height & width: {IMAGE_HEIGHT}x{IMAGE_WIDTH}
    Number of input channels: {in_channels}
    Number of output channels: {out_channels}
    Layers: {LAYERS}
    Batch size: {BATCH_SIZE}
    Learning rate: {LEARNING_RATE}
    Epochs: {EPOCHS}
    Early stopping patience: {EARLY_STOPPING_PATIENCE}
    Save interval: {SAVE_INTERVAL}
    Number of workers: {NUM_WORKERS}
    Pin memory: {PIN_MEMORY}
    Weight & Biases: {USE_WANDB}
    Deep supervision: {DEEP_SUPERVISION}''')

## Dataset

In [ ]:
def polar_transform(img, **kwargs):
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    value = np.sqrt(((width / 2.0) ** 2.0) + ((height / 2.0) ** 2.0))
    polar_img = cv.linearPolar(img, center, value, cv.WARP_FILL_OUTLIERS)
    polar_img = cv.rotate(polar_img, cv.ROTATE_90_COUNTERCLOCKWISE)
    return polar_img


def inverse_polar_transform(img, **kwargs):
    img = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
    height, width = img.shape[:2]
    center = (width // 2, height // 2)
    value = np.sqrt(((width / 2.0) ** 2.0) + ((height / 2.0) ** 2.0))
    cartesian_img = cv.linearPolar(img, center, value, cv.WARP_INVERSE_MAP | cv.WARP_FILL_OUTLIERS)
    return cartesian_img

In [ ]:
example_ds = OrigaDataset(IMAGE_DIR, MASK_DIR, os.listdir(IMAGE_DIR)[:1], A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=1.0),  # rotate by 0, 90, 180, or 270 degrees
    # A.Rotate(limit=30, p=0.33, border_mode=cv.BORDER_CONSTANT),
    # A.Lambda(image=polar_transform, mask=polar_transform),
    # A.Lambda(image=inverse_polar_transform, mask=inverse_polar_transform),

    # A.CLAHE(p=0.5),
    # A.RandomBrightnessContrast(p=0.5),
    # A.GridDistortion(p=0.5, border_mode=cv.BORDER_CONSTANT),
    # A.MedianBlur(p=0.5),
    # A.RandomToneCurve(p=0.5),
    # A.MultiplicativeNoise(p=0.5),
    ToTensorV2(),
]))
example_loader = DataLoader(example_ds, batch_size=1, shuffle=True)

example_image, example_mask = next(iter(example_loader))
print(f'Image shape: {example_image.shape}')
print(f'Mask shape: {example_mask.shape}')

unique, counts = np.unique(example_mask, return_counts=True)
print(f'Unique values and their counts in mask: {dict(zip(unique, counts))}')

# Plot example augmented images and masks
fig, ax = plt.subplots(3, 6, figsize=(12, 6))
ax = ax.ravel()
for i in range(0, 3 * 6, 2):
    batch = next(iter(example_loader))
    images, masks = batch
    image, mask = images[0], masks[0]
    image = image.permute(1, 2, 0).numpy()
    mask = mask.numpy()
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i + 1].imshow(mask, cmap='gray')
    ax[i + 1].axis('off')
plt.show()

In [ ]:
train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=1.0),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    ToTensorV2(),
])

train_loader, val_loader, test_loader = load_origa(
    IMAGE_DIR, MASK_DIR, 0.7, 0.15, 0.15,
    train_transform, val_transform, val_transform, BATCH_SIZE, PIN_MEMORY, NUM_WORKERS
)


def arctan(x):
    return 1e-7 + (1 - 2 * 1e-7) * (0.5 + torch.arctan(x) / torch.tensor(np.pi))


## Model

In [ ]:
# initialize model, loss, optimizer, scheduler, scaler, ...
model = Unet(in_channels, out_channels, LAYERS).to(DEVICE)
# model = UnetPlusPlus(in_channels, out_channels, LAYERS, deep_supervision=DEEP_SUPERVISION).to(DEVICE)
# model = Unet3Plus(in_channels, out_channels, LAYERS, deep_supervision=DEEP_SUPERVISION).to(DEVICE)

# model = AttentionUnet(in_channels, out_channels, LAYERS).to(DEVICE)
# model = InceptionUnet(in_channels, out_channels, LAYERS).to(DEVICE)

# model = ResUnet(in_channels, out_channels, LAYERS).to(DEVICE)
# model = RUnet(in_channels, out_channels, LAYERS).to(DEVICE)
# model = R2Unet(in_channels, out_channels, LAYERS).to(DEVICE)

# model = SqueezeUnet(in_channels, out_channels, LAYERS).to(DEVICE)

# model = R2AttentionUnet(in_channels, out_channels, LAYERS).to(DEVICE)
# model = R2UnetPlusPlus(in_channels, out_channels, LAYERS).to(DEVICE)

# model = ResAttentionUnetPlusPlus(in_channels, out_channels, LAYERS).to(DEVICE)
# model = RefUnet3PlusCBAM(in_channels, out_channels, LAYERS).to(DEVICE)
# model = DualRAUnetPlusPlus(in_channels, out_channels, LAYERS).to(DEVICE)
# model = DualRefUnet3PlusCBAM(in_channels, out_channels, LAYERS).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# criterion = nn.CrossEntropyLoss()
criterion = DiceLoss(out_channels)
# criterion = GeneralizedDice(out_channels)
# criterion = IoULoss(out_channels)
# criterion = FocalLoss(out_channels)
# criterion = TverskyLoss(out_channels)
# criterion = FocalTverskyLoss(out_channels)
# criterion = HausdorffLoss(out_channels)
# criterion = BoundaryLoss(out_channels)
# criterion = CrossEntropyLoss(out_channels)
# criterion = SensitivitySpecificityLoss(out_channels)
# criterion = EdgeLoss(out_channels)
# criterion = ComboLoss(out_channels)

# scheduler = None
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, verbose=True)

scaler = None
# scaler = torch.cuda.amp.GradScaler()


## Training

In [ ]:
# hist = train_multiclass(
#     model, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
#     save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
#     log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
#     save_best_model=False, plot_examples='none',
# )

hist = train_binary(
    model, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
    save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
    log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
    save_best_model=False, plot_examples='none',
)

# model = Unet(3, 1, LAYERS).to(DEVICE)
# checkpoint = torch.load(CHECKPOINT_DIR + 'binary-model.pth')
# model.load_state_dict(checkpoint)
# model2 = RefUnet3PlusCBAM(3, 1, LAYERS, model).to(DEVICE)
# optimizer = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
# hist = train_cascade(
#     model, model2, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
#     save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
#     log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
#     save_best_model=False, plot_examples='none',
# )

# model = DualRefUnet3PlusCBAM(3, 1, LAYERS).to(DEVICE)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# hist = train_dual(
#     model, criterion, criterion, optimizer, EPOCHS, DEVICE, train_loader, val_loader, scheduler, scaler,
#     save_interval=SAVE_INTERVAL, early_stopping_patience=EARLY_STOPPING_PATIENCE,
#     log_to_wandb=USE_WANDB, log_dir=LOGS_DIR, log_interval=LOG_INTERVAL, checkpoint_dir=CHECKPOINT_DIR,
#     save_best_model=False, plot_examples='none',
# )

In [ ]:
# Plot metrics
used_metrics = sorted([m[6:] for m in hist.keys() if m.startswith('train_')])
fig, ax = plt.subplots(4, 4, figsize=(14, 8))
ax = ax.ravel()

for i, metric in enumerate(used_metrics):
    ax[i].plot(hist[f'train_{metric}'], label=f'train')
    ax[i].plot(hist[f'val_{metric}'], label=f'val')
    ax[i].set_title(metric[0].upper() + metric[1:].replace('_', ' '))
    if metric != 'loss':
        ax[i].set_ylim(top=1)
    ax[i].legend()

for ax in ax[len(used_metrics):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

## Testing

In [ ]:
results = evaluate(model, criterion, DEVICE, test_loader)

In [ ]:
plot_results_from_loader(test_loader, model, DEVICE, types='all', n_samples=4, save_path=f'{LOGS_DIR}/evaluation.png')

## Work in progress

In [ ]:
# torch.save(model.state_dict(), CHECKPOINT_DIR + 'model.pth')

checkpoint = torch.load(CHECKPOINT_DIR + 'model.pth')
model = Unet(in_channels, out_channels, LAYERS).to(DEVICE)
model.load_state_dict(checkpoint)

In [ ]:
num_classes = 1

model = Unet(in_channels=3, out_channels=num_classes).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss = DiceLoss(num_classes=num_classes)
# loss = GeneralizedDice(num_classes=num_classes)
# loss = IoULoss(num_classes=num_classes)
# loss = FocalLoss(num_classes=num_classes)
# loss = TverskyLoss(num_classes=num_classes)
# loss = FocalTverskyLoss(num_classes=num_classes)
# loss = HausdorffLoss(num_classes=num_classes)
# loss = BoundaryLoss(num_classes=num_classes)
# loss = CrossEntropy(num_classes=num_classes)
# loss = SensitivitySpecificityLoss(num_classes=num_classes)
# loss = EdgeLoss(num_classes=num_classes)
# loss = ComboLoss(num_classes=num_classes)

for epoch in range(5):
    acc_loss = 0
    for images, masks in val_loader:
        images = images.float().to(DEVICE)
        masks = masks.long().to(DEVICE)

        if num_classes == 1:
            masks = torch.where(masks > 0, torch.tensor(1).to(DEVICE), torch.tensor(0).to(DEVICE))

        outputs = model(images)
        loss_value = loss(outputs, masks)
        acc_loss += loss_value.item()

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    # plot example
    images = images.cpu().numpy()
    masks = masks.cpu().numpy()
    if num_classes > 1:
        probs = F.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()
    else:
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float().cpu().numpy().transpose(0, 2, 3, 1)

    _, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(images[0].transpose(1, 2, 0) / 255.0)
    ax[1].imshow(masks[0])
    ax[2].imshow(preds[0])
    plt.show()

    print(f'Epoch {epoch + 1} loss:', acc_loss / len(val_loader))


## TODO: find out wtf is going on with the model's output

In [ ]:
images, masks = next(iter(val_loader))

# change order of images
# new_order = [0, 1, 0, 3]
# images = images[new_order]
# masks = masks[new_order]

images = images.float().to(DEVICE)
masks = masks.long().to(DEVICE)

outputs = model(images)
probs = F.softmax(outputs, dim=1)
preds = torch.argmax(probs, dim=1)

# plot probs
_, ax = plt.subplots(4, 6, figsize=(18, 12))
for i in range(4):
    ax[i, 0].imshow(images[i].cpu().numpy().transpose(1, 2, 0) / 255.0)
    ax[i, 1].imshow(masks[i].cpu().numpy())
    ax[i, 2].imshow(probs[i, 0].cpu().detach().numpy())
    ax[i, 3].imshow(probs[i, 1].cpu().detach().numpy())
    ax[i, 4].imshow(probs[i, 2].cpu().detach().numpy())
    ax[i, 5].imshow(preds[i].cpu().detach().numpy())
plt.show()

In [ ]:
image = images[1]
mask = masks[1]
rotated = torch.cat(
    [torch.rot90(image, k=k, dims=(1, 2)).unsqueeze(0) for k in range(4)] + \
    [torch.rot90(torch.flip(image, dims=(1,)), k=k, dims=(1, 2)).unsqueeze(0) for k in range(4)], dim=0)
rotated = rotated.to(DEVICE)

rotated_mask = torch.cat(
    [torch.rot90(mask, k=k, dims=(0, 1)).unsqueeze(0) for k in range(4)] + \
    [torch.rot90(torch.flip(mask, dims=(0,)), k=k, dims=(0, 1)).unsqueeze(0) for k in range(4)], dim=0)

outputs = model(rotated)
probs = F.softmax(outputs, dim=1)
preds = torch.argmax(probs, dim=1)

# plot probs
fig, ax = plt.subplots(4, 6, figsize=(18, 12))
for i in range(4):
    ax[i, 0].imshow(rotated[i].cpu().numpy().transpose(1, 2, 0) / 255.0)
    ax[i, 1].imshow(rotated_mask[i].cpu().numpy())
    ax[i, 2].imshow(probs[i, 0].cpu().detach().numpy())
    ax[i, 3].imshow(probs[i, 1].cpu().detach().numpy())
    ax[i, 4].imshow(probs[i, 2].cpu().detach().numpy())
    ax[i, 5].imshow(preds[i].cpu().detach().numpy())
plt.show()


In [ ]:
output = model(image.unsqueeze(0).to(DEVICE))
probs = F.softmax(output, dim=1)
pred = torch.argmax(probs, dim=1)

# plot probs
fig, ax = plt.subplots(1, 6, figsize=(18, 3))
ax[0].imshow(image.cpu().numpy().transpose(1, 2, 0) / 255.0)
ax[1].imshow(mask.cpu().numpy())
ax[2].imshow(probs[0, 0].cpu().detach().numpy())
ax[3].imshow(probs[0, 1].cpu().detach().numpy())
ax[4].imshow(probs[0, 2].cpu().detach().numpy())
ax[5].imshow(pred[0].cpu().detach().numpy())
plt.show()

## Contour detection method

In [ ]:
from skimage import segmentation
from scipy.ndimage import distance_transform_edt

images, masks = next(iter(val_loader))
images = images.float().to(DEVICE)
masks = masks.long().to(DEVICE)

image = images[0].cpu().numpy().transpose(1, 2, 0) / 255.0
mask = masks[0].cpu().numpy()
prediction = np.zeros_like(mask)
prediction[16:112, 16:112] = 1
prediction[32:96, 32:96] = 2

boundaries = segmentation.find_boundaries(mask, mode='inner').astype(np.uint8)
marked = image.copy()
marked[boundaries == 1] = [0, 0, 0]
dist_map = distance_transform_edt(1 - boundaries)
dist_map = dist_map / dist_map.max()

_, ax = plt.subplots(2, 4, figsize=(15, 7))
ax[0, 0].imshow(mask)
ax[0, 1].imshow(boundaries)
ax[0, 2].imshow(dist_map)
ax[0, 3].imshow(dist_map.max() - dist_map)
ax[1, 0].imshow(prediction)
ax[1, 1].imshow(marked)
plt.show()

In [ ]:
model = Unet(in_channels=3, out_channels=1).to(DEVICE)
loss = DiceLoss(num_classes=1)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(5):
    acc_loss = 0
    for images, masks in val_loader:

        edge_masks = np.zeros((masks.shape[0], masks.shape[1], masks.shape[2]))
        for b in range(masks.shape[0]):
            mask = masks[b].cpu().numpy()
            boundaries = segmentation.find_boundaries(mask, mode='thick').astype(np.uint8)
            edge_masks[b] = boundaries
        masks = torch.from_numpy(edge_masks)

        images = images.float().to(DEVICE)
        masks = masks.long().to(DEVICE)

        outputs = model(images)
        loss_value = loss(outputs, masks)
        acc_loss += loss_value.item()

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    # plot example
    images = images.cpu().numpy()
    masks = masks.cpu().numpy()
    # probs = F.softmax(outputs, dim=1)
    # preds = torch.argmax(probs, dim=1).cpu().numpy()
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).float().cpu().numpy().transpose(0, 2, 3, 1)

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(images[0].transpose(1, 2, 0) / 255.0)
    ax[1].imshow(masks[0])
    ax[2].imshow(preds[0])
    plt.show()

    print(f'Epoch {epoch + 1} loss:', acc_loss / len(val_loader))
